## Data Pre-Processing

### Colab Mount

In [2]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/'Colab_Notebooks'/LLM-Project-DS/
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/LLM-Project-DS
images	notebooks  Project_Journal.md  README.md


In [ ]:
# !pip install datasets # Colab needs this...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00


### Etc.

In [ ]:
# Imports
import os
import pandas as pd
import datasets
from datasets import load_dataset

## YELP

Prep for RoBERTa-base

In [10]:
dataset = load_dataset('yelp_review_full')

dataset # View

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [13]:
# Inspect
dataset['train'][0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

In [14]:
# Check Features
dataset['train'].features

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars'], id=None),
 'text': Value(dtype='string', id=None)}

In [19]:
train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])

display(df_train.head()), display(df_test.head())

,label,text
0,4,dr. goldberg offers everything i look for in a...
1,1,"Unfortunately, the frustration of being Dr. Go..."
2,3,Been going to Dr. Goldberg for over 10 years. ...
3,3,Got a letter in the mail last week that said D...
4,0,I don't know what Dr. Goldberg was like before...


,label,text
0,0,I got 'new' tires from them and within two wee...
1,0,Don't waste your time. We had two different p...
2,0,All I can say is the worst! We were the only 2...
3,0,I have been to this restaurant twice and was d...
4,0,Food was NOT GOOD at all! My husband & I ate h...


(None, None)

## EDA

In [22]:
# Shape
train.shape, test.shape

((650000, 2), (50000, 2))

In [ ]:
train.info()

In [ ]:
# Check for missing values
print(train.isnull().sum())

label    0
text     0
dtype: int64


In [ ]:
# Check class distribution
print(train['label'].value_counts(normalize=True))

label
4    0.2
1    0.2
3    0.2
0    0.2
2    0.2
Name: proportion, dtype: float64


## Text Preprocessing

In [37]:
# Imports
import re
import string
import nltk

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [41]:
# Downloads
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [42]:
# Cleaning function
def clean_text(review):
    # Lowercasing
    review = review.lower()

    # Remove special characters except punctuation
    review = re.sub(r'[^a-zA-Z0-9.,!?\'\s]', '', review)

    # Tokenize
    tokens = word_tokenize(review)

    return " ".join(tokens)

> I'm not removing punctuations or stop words as it may affect word meanings, and therefore, the sentiment.

In [43]:
# Apply to train & test data
train['clean_text'] = train['text'].apply(clean_text)
test['clean_text'] = test['text'].apply(clean_text)

train.head()

,label,text,clean_text
0,4,dr. goldberg offers everything i look for in a...,dr. goldberg offers everything i look for in a...
1,1,"Unfortunately, the frustration of being Dr. Go...","unfortunately , the frustration of being dr. g..."
2,3,Been going to Dr. Goldberg for over 10 years. ...,been going to dr. goldberg for over 10 years ....
3,3,Got a letter in the mail last week that said D...,got a letter in the mail last week that said d...
4,0,I don't know what Dr. Goldberg was like before...,i do n't know what dr. goldberg was like befor...


> We'll tokenize the cleaned text to prepare it for input into SamLowe's `roberta-base-go-emotions` model.

In [ ]:
# Create the directory if it doesn't exist
output_path = "data/pre-processed/"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [46]:
# Save train and test... we're moving to another PC...
train.to_csv(output_path + 'train.csv', index=False)
test.to_csv(output_path + 'test.csv', index=False)

if output_path + 'train.csv':
    print("Train data saved successfully.")
if output_path + 'test.csv':
    print("Test data saved successfully.")

Train data saved successfully.
Test data saved successfully.


### Loading the Model and Tokenizer

In [ ]:
# Load Data
train = pd.read_csv(output_path + 'train.csv')
test = pd.read_csv(output_path + 'test.csv')

In [ ]:
# Imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
# Load SamLowe's GoEmotions model
model_name = "SamLowe/roberta-base-go_emotions"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)